In [2]:
from collections import defaultdict
from gensim import corpora
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\Philippa\Documents\GitHub\crystalline-mining\extracted_pdf_info.csv")

df.head()

,pdf_text,metadata
0,['NATURAL CALCIUM CARBONATE FOR BIOMEDICAL APP...,"{'Author': 'ismail - [2010]', 'Creator': 'Micr..."
1,['ProceedingsofMachineLearningResearchLEAVEUNS...,"{'Author': '', 'CreationDate': 'D:202404290022..."
2,['Preprint Advances in Chemical Physics Vol. 1...,"{'CreationDate': 'D:20060424095709Z', 'ModDate..."
3,['Enhancing Drug-Drug Interaction Extraction f...,"{'Author': '', 'CreationDate': 'D:201805160029..."
4,['BERTChem-DDI : Improved Drug-Drug Interactio...,"{'Author': '', 'CreationDate': 'D:202012230158..."


In [ ]:
from gensim import corpora, models
from gensim.corpora.dictionary import Dictionary
from pyLDAvis import gensim_models

['this', 'is', 'a', 'test']
